# リバーシを作る

## 仕様
- 必須機能
  - 以下に示すような画面例を持つリバーシをプレイできる。（リバーシのルールは検索して調べること）
  - 人vs人対戦
  - 入力はキーボードからの数字とアルファべット→input()?
  - ゲーム終了時に勝敗，白と黒の石の数を表示
- 任意機能
  - 人vs ＣＰＵモード、CPU vs CPUモードの追加
  - 対戦結果の外部ファイルへの出力
  - ゲーム途中のセーブ機能とロード機能
  - その他の機能を自由に追加して構わない

## メモ
枠をつくる(8*8)
中央2*2[4,4], [4,5], [5,4], [5,5]の
○●
●○←が必要

白と黒の石（○と●）
挟んだら色が反転する
相手の色を挟める場所しか置けない
挟めなかったらパス
全部埋まるor どっちも置けない→ゲーム終了

### 作る関数
1. 盤面を表示
2. 石を置く
3. 石が置けるかをチェック

1. board関数
   tablesize*tablesizeの配列をつくる（intにする）
   初期条件：[3][3],[4][4]に白，[3][4]，[4][3]に黒
3. display関数
   配列の各要素に白，黒，空白を割り当て8*8できれいに表示
4. check関数
   盤内においたか
   →周りの8マスに相手の石があるか
    ない→faulse
    ある→その直線上の石を調べる
        自分の石→faulse
        相手の石→loop
5. put関数
   置く座標と色を指定
   →board[x][y] = colorとする
            
    

In [41]:
import numpy as np

white = 1
black = -1
blank = 0
tablesize = 8

class Reversi(object):
    
    def __init__(self):
        self.cell = np.zeros((tablesize, tablesize))
        self.cell = self.cell.astype(int)
        self.cell[3][3] = 1
        self.cell[3][4] = -1
        self.cell[4][3] = -1
        self.cell[4][4] = 1
        self.current = black    # 置いたプレイヤー
        self.turn = 1
        
    def turn_change(self):
        self.current *= -1
    
    def display(self):
        print("-" * 30)
        for x in range(tablesize):
            for y in range(tablesize):
                if self.cell[x][y] == 0:
                    print("-", end="  ")
                elif self.cell[x][y] == 1:
                    print("○", end="  ")
                else:
                    print("●", end="  ")
            print()

    # def put(self, x, y):    # あとで1ターンの処理をまとめる
    #     x = int(input("xを入力"))
    #     y = int(input("yを入力"))
    #     color = int(input("1 or -1"))
        
    #     self.cell[x][y] = color
        
    # 10/30ここまで
    # ここから→プレイヤーを分ける，石を置けるかチェックする，石を反転する

    def range_check(self, x, y):    # 盤内チェック
        return 0 <= x < tablesize and 0 <= y < tablesize
    
    def put_check(self, x, y):  # おけるかチェック
        if self.range_check(x, y):
            if self.cell[x][y] == blank:
                if self.get_flip_stone(x, y):
                    return True
        print("おけない")
        return False
    
    # def flip_check(self, x, y, dx, dy): # 反転チェック
    #     if self.range_check(x+dx, y+dy):
    #         length = 0
    #         while self.cell[x+dx][y+dy] == -self.current:   # 相手の石があったら
    #             x += dx
    #             y += dy
    #             length += 1
    #             if self.cell[x+dx][y+dy] == self.current:   # 自分の石があったら
    #                 return True
    
    # def flip_check(self, x, y, dx, dy): # 挟める石があったらTrue
    #     serchx, serchy = x + dx, y + dy
    #     length = 0
    #     if not self.range_check(serchx, serchy):
    #         return False
    #     if self.cell[serchx][serchy] != -self.current:
    #         return False
    #     while self.range_check(serchx, serchy):
    #         if self.cell[serchx][serchy] == -self.current:
    #             serchx += dx
    #             serchy += dy
    #             length += 1
    #         elif self.cell[serchx][serchy] == self.current:
    #             return True
    #         else:
    #             return False
    #     return False
    
    def get_flip_stone(self, x, y):
        fliplist = []
        for dx in range(-1, 2):
            for dy in range(-1, 2):
                if dx == 0 and dy == 0:
                    continue
                temp_flip = []
                
                serchx, serchy = x + dx, y + dy
                while self.range_check(serchx, serchy) and self.cell[serchx][serchy] == -self.current:
                    fliplist.append((serchx, serchy))
                    serchx += dx
                    serchy += dy
                if temp_flip and self.range_check(serchx, serchy) and self.cell[serchx][serchy] == self.current:
                    fliplist.extend(temp_flip)
        return fliplist
    
    # def flip_stone(self, x, y):
    #     for dx in range(-1, 2):
    #         for dy in range(-1, 2):
    #             length = self.flip_check(x, y, dx, dy)
    #             if length > 0:
    #                 for l in range(length):
    #                     k = l+1
    #                     self.cell[x + dx*k][y + dy*k] *= -1
    
    def put(self, x, y):
        flippable = self.get_flip_stone(x, y)
        if not flippable:
            return False
        
        self.cell[x][y] = self.current
        
        for r,c in flippable:
            self.cell[r][c] = self.current
            
        self.turn_change()
        self.turn += 1
        return True
                
        
            
            


    
    
if __name__ == "__main__":
    board = Reversi()
    print("start game")
    board.put(2, 1)
    board.display()
# b = board(8)
# b[3][2] = 1
# print(b)


# display(b)

# # put追加hhhhhhhhhhh
# put()
# range_check()
# display(b)

start game
------------------------------
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  ○  ●  -  -  -  
-  -  -  ●  ○  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  


In [52]:
class ShortReversi:
    def __init__(self):
        self.dirs = [-10, -9, -8, -1, 1, 8, 9, 10]
        self.discs = ' - o x\n'
        self.board = [0 if i % 9 else 3 for i in range(91)]
        self.board[40] = self.board[50] = self.turn = 1
        self.board[41] = self.board[49] = 2

    def _check(self, board, move, flip=False):
        if not (ret := False) and not board[move]:
            for i in range(8):
                count = value = move + self.dirs[i]
                while board[value] == 3 - self.turn:
                    value += self.dirs[i]
                if count != value and board[value] == self.turn:
                    ret = value = move
                    while flip:
                        board[value] = self.turn
                        value += self.dirs[i]
                        if board[value] == self.turn:
                            break
        return ret

    def play(self, com1=False, com2=True):
        end = False
        while not (move := 0):
            for i in range(9, 82):
                if self._check(self.board, i, flip=False) and not move:
                    move = i
                print(self.discs[self.board[i]*2:][:2], end='')
            while move and not (end := False):
                if not com1 and self.turn == 1 or not com2 and self.turn == 2:
                    x, y = [int(i) for i in input().split()]
                    move = x + y * 9
                if self._check(self.board, move, flip=True):
                    break
            else:
                if end:
                    break
                end, _ = True, print('pass')
            self.turn = 3 - self.turn


if __name__ == '__main__':
    ShortReversi().play()



 - - - - - - - -
 - - - - - - - -
 - - - - - - - -
 - - - o x - - -
 - - - x o - - -
 - - - - - - - -
 - - - - - - - -
 - - - - - - - -

 - - - - - - - -
 - - - - - - - -
 - - - - o - - -
 - - - o o - - -
 - - - x o - - -
 - - - - - - - -
 - - - - - - - -
 - - - - - - - -

 - - - - - - - -
 - - - - - - - -
 - - - x o - - -
 - - - x o - - -
 - - - x o - - -
 - - - - - - - -
 - - - - - - - -
 - - - - - - - -


ValueError: not enough values to unpack (expected 2, got 0)